In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import BertModel, BertTokenizer

# Implemented by myself
from config import *
from data_processer import CSCDataset
from models import CombineBertModel, DecoderBaseRNN, DecoderTransformer

In [2]:
def cal_err(raw_sentence, pred_sentence, corr_sentence):
    matrices = ["over_corr", "total_err", "true_corr"]
    char_level = {key: 0 for key in matrices}
    sent_level = {key: 0 for key in matrices}

    f1 = f2 = 0
    for i, c in enumerate(raw_sentence):
        pc, cc = pred_sentence[i], corr_sentence[i]

        if cc != c:
            char_level["total_err"] += 1
            char_level["true_corr"] += pc == cc
            f1 = 1
        elif pc != cc:
            char_level["over_corr"] += 1
            f2 = 1

    if f1:
        sent_level["true_corr"] += all(pred_sentence == corr_sentence)
        sent_level["total_err"] += f1
    sent_level["over_corr"] += f2

    return char_level, sent_level

In [3]:
def test(model, tokenizer, test_data_loader):
    model.eval()
    total_loss = 0
    matrices = ["over_corr", "total_err", "true_corr"]
    test_char_level = {key: 0 for key in matrices}
    test_sent_level = {key: 0 for key in matrices}

    with torch.no_grad():
        for batch in test_data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)

            loss = F.cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()

            t = torch.argmax(outputs, dim=-1)
            nt = t * attention_mask
            pred = tokenizer.batch_decode(nt, skip_special_tokens=True)

            for i in range(len(t)):
                char_level, sent_level = cal_err(input_ids[i], nt[i], labels[i])
                test_char_level = {
                    key: test_char_level[key] + v for key, v in char_level.items()
                }
                test_sent_level = {
                    key: test_sent_level[key] + v for key, v in sent_level.items()
                }
        print(total_loss / len(test_data_loader), test_char_level, test_sent_level)

In [4]:
tokenizer = BertTokenizer.from_pretrained(checkpoint)

In [5]:
train_dataset = CSCDataset([SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer)
train_data_loader = DataLoader(
    train_dataset, num_workers=0, shuffle=True, batch_size=16
)

test_dataset = CSCDataset([SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer)
test_data_loader = DataLoader(test_dataset, num_workers=0, shuffle=True, batch_size=16)

preprocessing sighan dataset: 2339it [00:00, 749635.29it/s]
preprocessing sighan dataset: 100%|███████████████████████████████████████████| 2339/2339 [00:00<00:00, 1168331.20it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 957607.47it/s]
preprocessing sighan dataset: 100%|███████████████████████████████████████████| 3437/3437 [00:00<00:00, 1147299.87it/s]

共3437句，共170330字，最长的句子有258字


In [6]:
def train(
    model, tokenizer, train_data_loader, test_data_loader=None, has_softmax=False
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(
            enumerate(train_data_loader),
            desc=f"Epoch:{epoch+1}/{epochs}",
            total=len(train_data_loader),
        )

        for i, batch in progress_bar:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

            # 反向传播在这，故labels不需要传入模型
            loss = F.cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

        with torch.no_grad():
            t = torch.argmax(outputs, dim=-1) if not has_softmax else outputs
            nt = t * attention_mask
            pred = tokenizer.batch_decode(nt, skip_special_tokens=True)
        # print(pred)
        # print(f"origin{tokenizer.batch_decode(labels, skip_special_tokens=True)}")

        for i, v in enumerate(nt):
            r, l = input_ids[i], labels[i]
            print(tokenizer.decode(r, skip_special_tokens=True))
            print(tokenizer.decode(v, skip_special_tokens=True))
            print(tokenizer.decode(l, skip_special_tokens=True))
            print(cal_err(r, v, l))

        print(f"Epoch {epoch+1} Loss: {total_loss / len(train_data_loader)}")

        # dev
        if test_data_loader:
            test(model, tokenizer, test_data_loader)

        # 释放不必要的内存
        # del input_ids, attention_mask, labels, outputs, loss
        # torch.cuda.empty_cache()
        # gc.collect()

#### BERT + LSTM
**长度不一致问题**：预测的时候有些字符预测为了 **[SEP]**

In [7]:
epochs = 5

# tokenizer = BertTokenizer.from_pretrained(checkpoint)
encoder_model = BertModel.from_pretrained(checkpoint)

# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

decoder_model = DecoderBaseRNN(
    model=nn.LSTM,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

train(model, tokenizer, train_data_loader)

Epoch:1/5:  42%|████████████████████████▍                                 | 62/147 [00:48<01:06,  1.28it/s, loss=6.152]


KeyboardInterrupt: 

#### BERT + GRU

In [ ]:
encoder_model = BertModel.from_pretrained(checkpoint)

# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

decoder_model = DecoderBaseRNN(
    model=nn.GRU,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

train(model, tokenizer, train_data_loader)

#### BERT + transformers

In [8]:
encoder_model = BertModel.from_pretrained(checkpoint)

nhead = 2
num_encoder_layers = 2
num_decoder_layers = 2

decoder_model = DecoderTransformer(
    input_size=encoder_model.config.hidden_size,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

train(model, tokenizer, train_data_loader)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
